In [1]:
from graph_utils import networkx, extract_features_from_nx_graph

g_nx_lbnl = networkx("D:\Master\Thesis\Code\LBNL_FDD_Dataset_FCU\LBNL_Ontology_Original.ttl")

In [2]:
lbnl_features = extract_features_from_nx_graph(
    g_nx_lbnl,
   "https://brickschema.org/schema/Brick#",
    node_type_filter=None,  # Or full URI if you want
    attribute_keys= ["type"],
    edge_predicates=["hasPoint"]
)

In [3]:
lbnl_features

{'FCU_RAT': {'type': 'Return_Air_Temperature_Sensor'},
 'Outdoor_air_damper': {'type': 'Outside_Damper',
  'hasPoint': ['FCU_DMPR', 'FCU_DMPR_DM']},
 'FCU_DMPR': {'type': 'Damper_Position_Sensor'},
 'FCU_RA_HUMD': {'type': 'Return_Air_Humidity_Sensor'},
 'FCU_DMPR_DM': {'type': 'Damper_Position_Command'},
 'Heating_coil': {'type': 'Hot_Water_Coil',
  'hasPoint': ['FCU_HVLV_DM',
   'FCU_HTG_RWT',
   'FCU_HVLV',
   'FCU_HTG_EWT',
   'FCU_HTG_GPM']},
 'FCU_HVLV_DM': {'type': 'Valve_Command'},
 'FCU_OA_HUMD': {'type': 'Outside_Air_Humidity_Sensor'},
 'FCU': {'type': 'FCU',
  'hasPoint': ['FCU_MA_HUMD',
   'FCU_CTRL',
   'FCU_RA_HUMD',
   'FCU_RAT',
   'FCU_OAT',
   'FCU_DAT',
   'FCU_DA_HUMD',
   'FCU_OA_HUMD',
   'FCU_MAT',
   'FCU_OA_CFM',
   'FCU_DA_CFM']},
 'FCU_MA_HUMD': {'type': 'Mixed_Air_Humidity_Sensor'},
 'FAN_CTRL': {'type': 'Mode_Command'},
 'FCU_HTG_RWT': {'type': 'Leaving_Water_Temperature_Sensor'},
 'FCU_WAT': {'type': 'Electricial_Meter'},
 'Cooling_coil': {'type': 'Chilled

### Convert data into dataframe, by type of sensor and component

In [4]:
import pandas as pd

# Optional: List of non-sensor types to exclude (extendable)
NON_SENSOR_TYPES = {
    "FCU", "Fan", "Chilled_Water_Coil", "Hot_Water_Coil", "Outside_Damper", "Zone"
}

def flatten_semantic_dict(sem_dict):
    # First, map sensor -> component
    sensor_to_component = {}
    for component, data in sem_dict.items():
        if 'hasPoint' in data:
            for sensor in data['hasPoint']:
                sensor_to_component[sensor] = component

    # Then, extract actual sensors (exclude known components)
    rows = []
    for sensor, data in sem_dict.items():
        if 'type' in data and data['type'] not in NON_SENSOR_TYPES:
            rows.append({
                'sensor': sensor,
                'type': data['type'],
                'component': sensor_to_component.get(sensor)
            })

    return pd.DataFrame(rows)

In [5]:
def clean_columns(df):
    for col in ['type', 'component']:
        df[col] = df[col].apply(lambda x: x[0] if isinstance(x, list) else x)
    return df

In [6]:
semantic_df = flatten_semantic_dict(lbnl_features)
semantic_df = clean_columns(semantic_df)
semantic_df['component'] = semantic_df['component'].fillna("Unknown")

In [7]:
semantic_df

,sensor,type,component
0,FCU_RAT,Return_Air_Temperature_Sensor,FCU
1,FCU_DMPR,Damper_Position_Sensor,Outdoor_air_damper
2,FCU_RA_HUMD,Return_Air_Humidity_Sensor,FCU
3,FCU_DMPR_DM,Damper_Position_Command,Outdoor_air_damper
4,FCU_HVLV_DM,Valve_Command,Heating_coil
5,FCU_OA_HUMD,Outside_Air_Humidity_Sensor,FCU
6,FCU_MA_HUMD,Mixed_Air_Humidity_Sensor,FCU
7,FAN_CTRL,Mode_Command,Supply_air_fan
8,FCU_HTG_RWT,Leaving_Water_Temperature_Sensor,Heating_coil
9,FCU_WAT,Electricial_Meter,Supply_air_fan


### Simplify sensor type

In [8]:
def simplify_type(type_str):
    if "Air_Temperature" in type_str or ("Air" in type_str and "Temperature" in type_str):
        return "Air_temp"
    elif "Water_Temperature" in type_str:
        return "Water_temp"
    elif "Air_Flow" in type_str:
        return "Air_flow"
    elif "Water" and "Flow" in type_str:
        return "Water_flow"
    elif "Humidity" in type_str:
        return "Humidity"
    elif "Valve_Position" in type_str:
        return "Valve_position"
    else:
        return type_str

semantic_df_simplified = semantic_df.copy()
semantic_df_simplified["Simplified_Type"] = semantic_df["type"].apply(simplify_type)
semantic_df_simplified.drop(columns=["type"], inplace=True)

In [9]:
semantic_df_simplified

,sensor,component,Simplified_Type
0,FCU_RAT,FCU,Air_temp
1,FCU_DMPR,Outdoor_air_damper,Damper_Position_Sensor
2,FCU_RA_HUMD,FCU,Humidity
3,FCU_DMPR_DM,Outdoor_air_damper,Damper_Position_Command
4,FCU_HVLV_DM,Heating_coil,Valve_Command
5,FCU_OA_HUMD,FCU,Humidity
6,FCU_MA_HUMD,FCU,Humidity
7,FAN_CTRL,Supply_air_fan,Mode_Command
8,FCU_HTG_RWT,Heating_coil,Water_temp
9,FCU_WAT,Supply_air_fan,Electricial_Meter


### Select only those sensors available in the sensor measurements

In [10]:
available_columns = pd.read_csv("measurements_LBNL.csv").pivot(index='Datetime', columns='sensor_id', values='measurement').columns.tolist()
available_columns

['FCU_CLG_GPM',
 'FCU_CLG_RWT',
 'FCU_CTRL',
 'FCU_CVLV',
 'FCU_CVLV_DM',
 'FCU_DAT',
 'FCU_DA_CFM',
 'FCU_DA_HUMD',
 'FCU_DMPR',
 'FCU_DMPR_DM',
 'FCU_HTG_GPM',
 'FCU_HTG_RWT',
 'FCU_HVLV',
 'FCU_HVLV_DM',
 'FCU_MAT',
 'FCU_MA_HUMD',
 'FCU_OAT',
 'FCU_OA_CFM',
 'FCU_OA_HUMD',
 'FCU_RAT',
 'FCU_RA_HUMD',
 'FCU_SPD',
 'FCU_WAT',
 'RMCLGSPT',
 'RMHTGSPT',
 'RM_TEMP']

In [11]:
semantic_df_simplified = semantic_df_simplified[semantic_df_simplified['sensor'].isin(available_columns)]

In [12]:
semantic_df_simplified

,sensor,component,Simplified_Type
0,FCU_RAT,FCU,Air_temp
1,FCU_DMPR,Outdoor_air_damper,Damper_Position_Sensor
2,FCU_RA_HUMD,FCU,Humidity
3,FCU_DMPR_DM,Outdoor_air_damper,Damper_Position_Command
4,FCU_HVLV_DM,Heating_coil,Valve_Command
5,FCU_OA_HUMD,FCU,Humidity
6,FCU_MA_HUMD,FCU,Humidity
8,FCU_HTG_RWT,Heating_coil,Water_temp
9,FCU_WAT,Supply_air_fan,Electricial_Meter
10,FCU_CVLV_DM,Cooling_coil,Valve_Command


### We group by component

In [13]:
grouped_by_component = semantic_df_simplified.groupby("component")["sensor"].apply(list).reset_index()

In [14]:
# Increase column width and display size
pd.set_option('display.max_colwidth', None)
grouped_by_component.iloc[0]

component                                         Cooling_coil
sensor       [FCU_CVLV_DM, FCU_CVLV, FCU_CLG_RWT, FCU_CLG_GPM]
Name: 0, dtype: object

In [15]:
for component, sensors in grouped_by_component.itertuples(index=False):
    print(len(sensors))
    print(component, sensors)

4
Cooling_coil ['FCU_CVLV_DM', 'FCU_CVLV', 'FCU_CLG_RWT', 'FCU_CLG_GPM']
11
FCU ['FCU_RAT', 'FCU_RA_HUMD', 'FCU_OA_HUMD', 'FCU_MA_HUMD', 'FCU_DA_CFM', 'FCU_CTRL', 'FCU_DA_HUMD', 'FCU_DAT', 'FCU_OAT', 'FCU_OA_CFM', 'FCU_MAT']
4
Heating_coil ['FCU_HVLV_DM', 'FCU_HTG_RWT', 'FCU_HVLV', 'FCU_HTG_GPM']
2
Outdoor_air_damper ['FCU_DMPR', 'FCU_DMPR_DM']
2
Supply_air_fan ['FCU_WAT', 'FCU_SPD']
3
Zone ['RM_TEMP', 'RMHTGSPT', 'RMCLGSPT']
